In [6]:
import tkinter as tk
from tkinter import colorchooser, messagebox, scrolledtext
import toml
import os
import sys
import threading
import time
import requests
from io import BytesIO

SETTINGS_FILE = 'settings.toml'

class DraggableButton(tk.Button):
    def __init__(self, master, **kwargs):
        super().__init__(master, **kwargs)
        self.bind("<Button-1>", self.start_drag)
        self.bind("<B1-Motion>", self.do_drag)
        self.bind("<ButtonRelease-1>", self.stop_drag)
        self.start_x = 0
        self.start_y = 0

    def start_drag(self, event):
        self.start_x = event.x
        self.start_y = event.y

    def do_drag(self, event):
        x = self.winfo_x() - self.start_x + event.x
        y = self.winfo_y() - self.start_y + event.y
        self.place(x=x, y=y)

    def stop_drag(self, event):
        pass

class DesktopApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Python Desktop")
        self.root.geometry("800x600")
        self.bg_color = "lightgray"
        self.taskbar_color = "gray"
        self.load_settings()
        self.root.configure(bg=self.bg_color)
        self.create_taskbar()
        self.create_shortcuts()
        self.create_shell()

    def create_taskbar(self):
        self.taskbar = tk.Frame(self.root, bg=self.taskbar_color, height=40)
        self.taskbar.pack(side=tk.BOTTOM, fill=tk.X)
        self.start_button = tk.Button(self.taskbar, text="Start", command=self.toggle_start_menu)
        self.start_button.pack(side=tk.LEFT, padx=10)
        self.start_menu = tk.Menu(self.root, tearoff=0)
        self.start_menu.add_command(label="Customize Desktop", command=self.open_settings)
        self.start_menu.add_command(label="App 1", command=lambda: self.open_app("App 1"))
        self.start_menu.add_command(label="App 2", command=lambda: self.open_app("App 2"))
        self.start_menu.add_command(label="Shell", command=self.toggle_shell)

    def toggle_start_menu(self):
        try:
            self.start_menu.post(self.root.winfo_pointerx(), self.root.winfo_pointery())
        finally:
            self.start_menu.grab_release()

    def create_shortcuts(self):
        self.create_shortcut("App 1", self.open_app)
        self.create_shortcut("App 2", self.open_app)
        self.create_shortcut("Shell", self.toggle_shell)

    def create_shortcut(self, name, command):
        button = DraggableButton(self.root, text=name, command=lambda: command(name), width=20, height=2)
        button.place(x=50, y=50 + 60 * len(self.root.children))

    def create_shell(self):
        self.shell_frame = tk.Frame(self.root, bg="black", padx=10, pady=10)
        self.shell_frame.place(x=0, y=40, width=800, height=200)  # Adjust height and width as needed

        # Title bar with buttons
        self.title_bar = tk.Frame(self.shell_frame, bg="darkgray", height=30)
        self.title_bar.pack(fill=tk.X)

        # Close Button
        self.close_button = tk.Button(self.title_bar, text="X", command=self.close_shell, width=4, height=1, bg="red", fg="white")
        self.close_button.pack(side=tk.RIGHT)

        # Maximize Button
        self.maximize_button = tk.Button(self.title_bar, text="□", command=self.maximize_shell, width=4, height=1, bg="lightgray")
        self.maximize_button.pack(side=tk.RIGHT)

        # Minimize Button
        self.minimize_button = tk.Button(self.title_bar, text="_", command=self.minimize_shell, width=4, height=1, bg="lightgray")
        self.minimize_button.pack(side=tk.RIGHT)

        self.shell_output = scrolledtext.ScrolledText(self.shell_frame, wrap=tk.WORD, height=10, width=100, bg="black", fg="white")
        self.shell_output.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

        self.shell_input = tk.Entry(self.shell_frame, width=100, bg="black", fg="white")
        self.shell_input.pack(padx=10, pady=10)
        self.shell_input.bind("<Return>", self.execute_command)

        self.shell_frame.pack_forget()  # Initially hide the shell

        self.commands = {
            "echo": self.echo_command,
            "date": self.date_command,
            "dir": self.dir_command,
            "mkdir": self.mkdir_command,
            "rmdir": self.rmdir_command,
            "del": self.del_command,
            "type": self.type_command,
            "cls": self.cls_command,
            "copy": self.copy_command,
            "move": self.move_command,
            "wget": self.wget_command
        }

    def toggle_shell(self):
        if self.shell_frame.winfo_ismapped():
            self.shell_frame.pack_forget()
        else:
            self.shell_frame.pack(side=tk.BOTTOM, fill=tk.X, padx=5, pady=5)

    def minimize_shell(self):
        self.shell_frame.pack_forget()

    def maximize_shell(self):
        self.shell_frame.place(x=0, y=40, width=self.root.winfo_width(), height=self.root.winfo_height() - 40)

    def close_shell(self):
        self.shell_frame.pack_forget()

    def open_app(self, app_name):
        messagebox.showinfo("Application", f"{app_name} clicked!")

    def open_settings(self):
        settings_window = tk.Toplevel(self.root)
        settings_window.title("Settings")
        settings_window.geometry("300x200")

        tk.Label(settings_window, text="Desktop Background Color:").pack(pady=5)
        bg_color_button = tk.Button(settings_window, text="Choose Color", command=self.choose_bg_color)
        bg_color_button.pack(pady=5)

        tk.Label(settings_window, text="Taskbar Color:").pack(pady=5)
        taskbar_color_button = tk.Button(settings_window, text="Choose Color", command=self.choose_taskbar_color)
        taskbar_color_button.pack(pady=5)

        tk.Button(settings_window, text="Apply Settings", command=self.apply_settings).pack(pady=20)

    def choose_bg_color(self):
        color_code = colorchooser.askcolor(title="Choose Background Color")[1]
        if color_code:
            self.bg_color = color_code

    def choose_taskbar_color(self):
        color_code = colorchooser.askcolor(title="Choose Taskbar Color")[1]
        if color_code:
            self.taskbar_color = color_code

    def apply_settings(self):
        self.root.configure(bg=self.bg_color)
        self.taskbar.configure(bg=self.taskbar_color)
        self.save_settings()

    def save_settings(self):
        settings = {
            'desktop': {
                'background_color': self.bg_color,
                'taskbar_color': self.taskbar_color
            }
        }
        with open(SETTINGS_FILE, 'w') as f:
            toml.dump(settings, f)

    def load_settings(self):
        try:
            with open(SETTINGS_FILE, 'r') as f:
                settings = toml.load(f)
                self.bg_color = settings.get('desktop', {}).get('background_color', 'lightgray')
                self.taskbar_color = settings.get('desktop', {}).get('taskbar_color', 'gray')
        except FileNotFoundError:
            pass

    def execute_command(self, event):
        command = self.shell_input.get().strip()
        self.shell_output.insert(tk.END, f"> {command}\n")
        self.shell_input.delete(0, tk.END)
        
        parts = command.split()
        cmd_name = parts[0]
        args = parts[1:]

        if cmd_name in self.commands:
            result = self.commands[cmd_name](args)
        else:
            result = "Unknown command"
        
        self.shell_output.insert(tk.END, f"{result}\n")
        self.shell_output.yview(tk.END)

    def echo_command(self, args):
        return " ".join(args)

    def date_command(self, args):
        from datetime import datetime
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    def dir_command(self, args):
        return "\n".join(os.listdir(os.getcwd()))

    def mkdir_command(self, args):
        if args:
            directory = args[0]
            try:
                os.makedirs(directory, exist_ok=True)
                return f"Directory '{directory}' created."
            except Exception as e:
                return f"Error: {e}"
        return "Error: Missing directory name."

    def rmdir_command(self, args):
        if args:
            directory = args[0]
            try:
                os.rmdir(directory)
                return f"Directory '{directory}' removed."
            except Exception as e:
                return f"Error: {e}"
        return "Error: Missing directory name."

    def del_command(self, args):
        if args:
            file = args[0]
            try:
                os.remove(file)
                return f"File '{file}' deleted."
            except Exception as e:
                return f"Error: {e}"
        return "Error: Missing file name."

    def type_command(self, args):
        if args:
            file = args[0]
            try:
                with open(file, 'r') as f:
                    return f.read()
            except Exception as e:
                return f"Error: {e}"
        return "Error: Missing file name."

    def cls_command(self, args):
        self.shell_output.delete(1.0, tk.END)
        return "Screen cleared."

    def copy_command(self, args):
        if len(args) == 2:
            src, dest = args
            try:
                with open(src, 'rb') as fsrc:
                    with open(dest, 'wb') as fdst:
                        fdst.write(fsrc.read())
                return f"File '{src}' copied to '{dest}'."
            except Exception as e:
                return f"Error: {e}"
        return "Error: Incorrect number of arguments."

    def move_command(self, args):
        if len(args) == 2:
            src, dest = args
            try:
                os.rename(src, dest)
                return f"File '{src}' moved to '{dest}'."
            except Exception as e:
                return f"Error: {e}"
        return "Error: Incorrect number of arguments."

    def wget_command(self, args):
        if len(args) == 2:
            url, filename = args
            try:
                response = requests.get(url, stream=True)
                total_size = int(response.headers.get('content-length', 0))
                with open(filename, 'wb') as f:
                    downloaded = 0
                    for chunk in response.iter_content(chunk_size=1024):
                        if chunk:
                            f.write(chunk)
                            downloaded += len(chunk)
                            percent = (downloaded / total_size) * 100
                            self.shell_output.insert(tk.END, f"Downloading: {percent:.2f}%\n")
                            self.shell_output.update()
                return f"File '{filename}' downloaded successfully."
            except Exception as e:
                return f"Error: {e}"
        return "Error: Incorrect number of arguments."

if __name__ == "__main__":
    root = tk.Tk()
    app = DesktopApp(root)
    root.mainloop()
